In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
from utils import combine_csv_files_to_dataframe,create_directory, smooth_and_plot, smooth_and_compute_derivative

In [3]:
# CV pipes
from cv_dataloader import create_image_data_loader
from cv_model import create_model
from cv_train import save_cv_predictions
# Timeseries pipes
from model import TransformerModel_ver2
from dataloader import create_data_loader
from train import save_predictions

In [4]:
PATH_TO_DATA = "E:/DS/debug_siam/test/"

# Load data and make parquet

In [5]:
create_directory(PATH_TO_DATA,'preproc_ts')

In [6]:
create_directory(PATH_TO_DATA,'plots')

In [7]:
create_directory(PATH_TO_DATA,'preds')

In [8]:
df = combine_csv_files_to_dataframe(PATH_TO_DATA)

Error processing file 846291a2-6475-47ac-8f10-08eac6b93fcb: No columns to parse from file


In [9]:
df.to_parquet(Path(PATH_TO_DATA) / 'preproc_ts/source.parquet', index=False)

## preproc CV

In [28]:
test = pd.read_parquet(Path(PATH_TO_DATA) / 'preproc_ts/source.parquet')

In [30]:
#new_train = []
un_ids = test.file_name.unique()
errors = []
for uuid in tqdm(un_ids, total = len(un_ids)):
    #file_path = os.path.join('E:/DS/data/siam/bi/test/', uuid)
    #if os.path.isfile(file_path):
        # Read the CSV file without a header and with tab as the separator
        #try:
            #df = pd.read_csv(file_path, sep='\t', header=None)
    
            # Ensure the file has exactly 3 columns (time, atm, der_atm)
    
            # Assign column names to the DataFrame
            #df.columns = ['time', 'atm', 'der_atm']
    
            # Add a new column for the file name
            #df['file_name'] = uuid
    
            # Reorder columns to match the desired output format
            #df = df[['file_name', 'time', 'atm', 'der_atm']]
    cc = test[test.file_name==uuid].copy()
    try:
        smooth_and_plot(cc, Path(PATH_TO_DATA) / 'plots/', pressure_type='origin', derivative_type='origin', save_plot=True)
        #new_train.append(cc)
    except:
        errors.append(uuid)
        # except:
        #     errors.append(uuid)                

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [04:43<00:00,  1.76it/s]


## preproc + 7 feats
если файлов больше 1000 - для производительности код переписать, чтобы он работал с файлами исходными

In [10]:
test = df.copy()
un_ids = test.file_name.unique()
new_test = []
for uuid in tqdm(un_ids, total=len(un_ids)):
    cc = test[test.file_name==uuid]
    cc = smooth_and_compute_derivative(cc)
    new_test.append(cc)
new_test = pd.concat(new_test)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:10<00:00, 48.20it/s]


## preproc l1p

In [11]:
df[df.columns[1:]] = np.log1p(df[df.columns[1:]].abs())

In [12]:
new_test[new_test.columns[1:]] = np.log1p(new_test[new_test.columns[1:]].abs())

In [13]:
df.to_parquet(Path(PATH_TO_DATA) / 'preproc_ts/feat3_l1p.parquet', index=False)

In [14]:
new_test.to_parquet(Path(PATH_TO_DATA) / 'preproc_ts/feat10_l1p.parquet', index=False)

## preproc robustscaler

# load models

In [15]:
swin_v6 =  create_model(
    model_type = "swin",
    #model_name='b7',
    n_binary_targets = 8,
    n_regression_targets = 7,
    dropout = 0.1,
    freeze_backbone = False,
    use_multi_layer_features = False,
    pretrained = True,
    weights_path='./weights/swin_v6_l1p.pth'
)

In [16]:
vit_v4 =  create_model(
    model_type = "vit",
    #model_name='b7',
    n_binary_targets = 8,
    n_regression_targets = 7,
    dropout = 0.1,
    freeze_backbone = False,
    use_multi_layer_features = False,
    pretrained = True,
    weights_path='./weights/vit_v4_l1p.pth'
)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
conv_v5 =  create_model(
    model_type = "convnext",
    #model_name='b7',
    n_binary_targets = 8,
    n_regression_targets = 7,
    dropout = 0.1,
    freeze_backbone = False,
    use_multi_layer_features = False,
    pretrained = True,
    weights_path='./weights/conv_v5_l1p.pth'
)

In [18]:
model_ts10 = TransformerModel_ver2(
    input_dim=10,
    d_model=256,
    nhead=4,
    num_encoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len = 1024
)
model_ts10.load_state_dict(torch.load("./weights/ts_10f_l1p_1.pth"))
model_ts10.eval()

TransformerModel_ver2(
  (input_projection): Linear(in_features=10, out_features=256, bias=True)
  (blocks): ModuleList(
    (0-3): 4 x TransformerBlock(
      (norm1): RMSNorm()
      (attn): MultiHeadAttention(
        (q_proj): Linear(in_features=256, out_features=256, bias=True)
        (k_proj): Linear(in_features=256, out_features=256, bias=True)
        (v_proj): Linear(in_features=256, out_features=256, bias=True)
        (out_proj): Linear(in_features=256, out_features=256, bias=True)
        (rope): RotaryPositionalEmbedding()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): RMSNorm()
      (mlp): SwiGLU(
        (w1): Linear(in_features=256, out_features=512, bias=True)
        (w2): Linear(in_features=256, out_features=512, bias=True)
        (w3): Linear(in_features=512, out_features=256, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (norm): RMSNorm()
  (binary_output): Linear(in_features=256, out_features=8, bias=True

In [19]:
model_ts3 = TransformerModel_ver2(
    input_dim=3,
    d_model=256,
    nhead=4,
    num_encoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len = 1024
)
model_ts3.load_state_dict(torch.load("./weights/ts_3f_l1p_1.pth"))
model_ts3.eval()

TransformerModel_ver2(
  (input_projection): Linear(in_features=3, out_features=256, bias=True)
  (blocks): ModuleList(
    (0-3): 4 x TransformerBlock(
      (norm1): RMSNorm()
      (attn): MultiHeadAttention(
        (q_proj): Linear(in_features=256, out_features=256, bias=True)
        (k_proj): Linear(in_features=256, out_features=256, bias=True)
        (v_proj): Linear(in_features=256, out_features=256, bias=True)
        (out_proj): Linear(in_features=256, out_features=256, bias=True)
        (rope): RotaryPositionalEmbedding()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): RMSNorm()
      (mlp): SwiGLU(
        (w1): Linear(in_features=256, out_features=512, bias=True)
        (w2): Linear(in_features=256, out_features=512, bias=True)
        (w3): Linear(in_features=512, out_features=256, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (norm): RMSNorm()
  (binary_output): Linear(in_features=256, out_features=8, bias=True)

# prepare dataloaders

In [20]:
cv_loader  = create_image_data_loader(
    csv_path=None,
    images_dir=Path(PATH_TO_DATA) / 'plots/',
    batch_size = 64,
    shuffle = False,
    num_workers = 0
)

In [21]:
test_loader_10f  = create_data_loader(
    csv_path=None,
    parquet_path=Path(PATH_TO_DATA) / 'preproc_ts/feat10_l1p.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)

Preprocessing time series data...


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2304.29it/s]


Preprocessed 500 time series


In [22]:
test_loader_3f  = create_data_loader(
    csv_path=None,
    parquet_path=Path(PATH_TO_DATA) / 'preproc_ts/feat3_l1p.parquet',
    batch_size=64,
    max_seq_len=1024,
    num_workers=0
)


Preprocessing time series data...


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2475.26it/s]

Preprocessed 500 time series


# make preds

In [23]:
save_cv_predictions(swin_v6,cv_loader,
                Path(PATH_TO_DATA)/'preds/inf_CV_swin_v6_l1p.csv',
                'cuda',
                binary_threshold=0.5)

Running inference: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.17s/it]

Predictions saved to E:\DS\debug_siam\test\preds\inf_CV_swin_v6_l1p.csv


In [24]:
save_cv_predictions(vit_v4,cv_loader,
                Path(PATH_TO_DATA)/'preds/inf_CV_vit_v4_l1p.csv',
                'cuda',
                binary_threshold=0.5)

Running inference: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.14s/it]

Predictions saved to E:\DS\debug_siam\test\preds\inf_CV_vit_v4_l1p.csv


In [25]:
save_cv_predictions(conv_v5,cv_loader,
                Path(PATH_TO_DATA)/'preds/inf_CV_conv_v5_l1p.csv',
                'cuda',
                binary_threshold=0.5)

Running inference: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.13s/it]

Predictions saved to E:\DS\debug_siam\test\preds\inf_CV_conv_v5_l1p.csv


In [26]:
save_predictions(model_ts3,test_loader_3f,
                Path(PATH_TO_DATA)/'preds/inf_l1p_ts3_v1.csv',
                'cuda',
                binary_threshold=0.5)

Running inference: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  7.57it/s]

Predictions saved to E:\DS\debug_siam\test\preds\inf_l1p_ts3_v1.csv


In [27]:
save_predictions(model_ts10,test_loader_10f,
                Path(PATH_TO_DATA)/'preds/inf_l1p_ts10_v1.csv',
                'cuda',
                binary_threshold=0.5)

Running inference: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00,  8.98it/s]

Predictions saved to E:\DS\debug_siam\test\preds\inf_l1p_ts10_v1.csv


# blending for final result and save

In [28]:
keep_cols = ['file_name',
       'binary_Некачественное ГДИС', 
       'binary_Влияние ствола скважины', 
       'binary_Радиальный режим',
       'binary_Линейный режим', 
       'binary_Билинейный режим', 
       'binary_Сферический режим', 
       'binary_Граница постоянного давления',
       'binary_Граница непроницаемый разлом',
       'regression_Влияние ствола скважины_details',
       'regression_Радиальный режим_details',
       'regression_Линейный режим_details',
       'regression_Билинейный режим_details',
       'regression_Сферический режим_details',
       'regression_Граница постоянного давления_details',
       'regression_Граница непроницаемый разлом_details']
columns_b=['Некачественное ГДИС',
       'Влияние ствола скважины', 'Радиальный режим', 'Линейный режим',
       'Билинейный режим', 'Сферический режим', 'Граница постоянного давления',
       'Граница непроницаемый разлом', ]
columns_r=['Влияние ствола скважины_details',
       'Радиальный режим_details', 'Линейный режим_details',
       'Билинейный режим_details', 'Сферический режим_details',
       'Граница постоянного давления_details',
       'Граница непроницаемый разлом_details']

columns_b_2 = ['binary_Некачественное ГДИС', 
       'binary_Влияние ствола скважины', 
       'binary_Радиальный режим',
       'binary_Линейный режим', 
       'binary_Билинейный режим', 
       'binary_Сферический режим', 
       'binary_Граница постоянного давления',
       'binary_Граница непроницаемый разлом',]
columns_r_2 =['regression_Влияние ствола скважины_details',
       'regression_Радиальный режим_details',
       'regression_Линейный режим_details',
       'regression_Билинейный режим_details',
       'regression_Сферический режим_details',
       'regression_Граница постоянного давления_details',
       'regression_Граница непроницаемый разлом_details']

In [29]:
# TS_3 l1p v1 Результат: 0.6881 with weights
ch8 =pd.read_csv(Path(PATH_TO_DATA)/'preds/inf_l1p_ts3_v1.csv').sort_values('file_name')
ch8[columns_r_2] = np.expm1(ch8[columns_r_2])

In [33]:
# TS 10   l1p  Результат: 0.6866 with weights  inf_l1p_ts10_v1
ch10 = pd.read_csv(Path(PATH_TO_DATA)/'preds/inf_l1p_ts10_v1.csv').sort_values('file_name')
ch10[columns_r_2] = np.expm1(ch10[columns_r_2])

In [35]:
# VS Swin l1p  Результат: 0.6189 with weights
ch11 = pd.read_csv(Path(PATH_TO_DATA)/'preds/inf_CV_swin_v6_l1p.csv').sort_values('file_name')
ch11[columns_r_2] = np.expm1(ch11[columns_r_2])

In [36]:
# VS Conv l1p v1 Результат: 0.5541 with weights
ch9 = pd.read_csv( Path(PATH_TO_DATA)/'preds/inf_CV_conv_v5_l1p.csv').sort_values('file_name')
ch9[columns_r_2] = np.expm1(ch9[columns_r_2])

In [37]:
# VS vit l1p v4 Результат: Результат: 0.517 with weights
ch12 = pd.read_csv(Path(PATH_TO_DATA)/'preds/inf_CV_vit_v4_l1p.csv').sort_values('file_name')
ch12[columns_r_2] = np.expm1(ch12[columns_r_2])

In [38]:
def blend_them(dfs_b, dfs_r):
    # for df in dfs_b+dfs_r:
    #     df = df.sort_values('file_name')
    # for df in dfs_r:
    #     df = apply_trans_reg(df, path_to_scalers='E:/DS/data/siam/scalers/tg_scalers_r/tg_r_')
    res = dfs_b[0][['file_name']].copy()
    for col in columns_r_2:
        base = dfs_r[0][col].values/len(dfs_r)
        if len(dfs_r)>1:
            for df in dfs_r[1:]:
                base += df[col].values/len(dfs_r)
        res[col]=base
    for col in columns_b_2:
        col_ = col
        col =col+'_prob'
        base = dfs_b[0][col].values/len(dfs_b)
        if len(dfs_b)>1:
            for df in dfs_b[1:]:
                base += df[col].values/len(dfs_b)
        res[col_]=((base>0.5)*1)
    res = res[keep_cols]
    res.columns = ['file_name']+columns_b+columns_r
    return res

In [39]:
#Результат: 0.7297
res3_2 =  blend_them(dfs_b=[ch8,ch12, ch9,ch10,ch11], dfs_r=[ch8,ch10])

In [40]:
create_directory(PATH_TO_DATA,'FINAL')

In [41]:
res3_2.to_csv(Path(PATH_TO_DATA)/'FINAL/blend3_2.csv',index=False)